# **Kaggle Playground Series - Season 5, Episode 2**

## **Objective**  
In this episode we are given the task of predicting the price of backpacks given various attributes.  
Submissions are scored on the root mean squared error.  

RMSE is defined as:

$$
\textrm{RMSE} =  \left( \frac{1}{N} \sum_{i=1}^{N} (y_i - \widehat{y}_i)^2 \right)^{\frac{1}{2}}
$$

## **Data**  
The dataset for this competition is generated from a deep learning model trained on the [Student Bag Pric Preditions Dateset](https://www.kaggle.com/datasets/souradippal/student-bag-price-prediction-dataset/data)


In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import optuna
from optuna.study import MaxTrialsCallback

print('Libaires imported')

In [ ]:
df_train = pd.read_csv('/kaggle/input/playground-series-s5e2/train.csv')
df_train_ex = pd.read_csv('/kaggle/input/playground-series-s5e2/training_extra.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s5e2/test.csv')
df_sub = pd.read_csv('/kaggle/input/playground-series-s5e2/sample_submission.csv')
print('Data imported')

### **Initial Observations**  


In [ ]:
df_test

In [ ]:
df_train

In [ ]:
df_train_ex

In [ ]:
df_train_combined = pd.concat([df_train, df_train_ex], ignore_index=True)

In [ ]:
df_train_combined

In [ ]:
df_train = df_train_combined

### **Summary of Datasets**

In [ ]:
missing_values_train = pd.DataFrame({'Feature': df_train.columns,
                              '[TRAIN] No. of Missing Values': df_train.isnull().sum().values,
                              '[TRAIN] % of Missing Values': ((df_train.isnull().sum().values)/len(df_train)*100)})

missing_values_test = pd.DataFrame({'Feature': df_test.columns,
                             '[TEST] No.of Missing Values': df_test.isnull().sum().values,
                             '[TEST] % of Missing Values': ((df_test.isnull().sum().values)/len(df_test)*100)})

unique_values = pd.DataFrame({'Feature': df_train.columns,
                              'No. of Unique Values[FROM TRAIN]': df_train.nunique().values})

feature_types = pd.DataFrame({'Feature': df_train.columns,
                              'DataType': df_train.dtypes})

df_summary = pd.merge(missing_values_train, missing_values_test, on='Feature', how='left')
df_summary = pd.merge(df_summary, unique_values, on='Feature', how='left')
df_summary = pd.merge(df_summary, feature_types, on='Feature', how='left')

df_summary

In [ ]:
train_duplicates = df_train.duplicated().sum()
test_duplicates = df_test.duplicated().sum()

print(f"Number of duplicate rows in df_train: {train_duplicates}")
print(f"Number of duplicate rows in df_test: {test_duplicates}")

## **Dataset Observations**

### **Shape**
Training Data: 3,994,318rows × 11 columns  
Test Data: 200,000 rows × 10 columns

### **Missing Values**
Several features contain missing values in both the training and test sets:  

**Training set**
- Brand: ~3.17%
- Material:  ~2.78%
- Size: ~2.20%
- Laptop Compartment: ~2.47%
- Waterproof: ~2.36%
- Style: ~2.61%
- Color: ~3.35%
- Weight Capacity (kg): ~0.05%

**Test set**
- Brand: ~3.11%
- Material: ~2.81%
- Size: ~2.19%
- Laptop Compartment: ~2.48%
- Waterproof: ~2.41%
- Style: ~2.58%
- Color: ~3.39%
- Weight Capacity (kg): ~0.04%


### **Feature Breakdown**

- **ID:** A unique identifier for each backpack.
- **Brand, Material, Size, Style:** Categorical variables.
- **Compartments:** Numeric, range from 1 to 10.
- **Laptop Compartment & Waterproof:** Binary categorical.
- **Color:** 6 unique values, with missing data.
- **Weight Capacity (kg):** Numerical.
- **Price:** The target variable in the training set.

## **EDA**

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(df_train["Price"], bins=50, kde=True)
plt.xlabel("Price")
plt.ylabel("Frequency")
plt.title("Distribution of Price (Target Variable)")
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(df_train.isnull(), cbar=False, cmap="viridis")
plt.title("Missing Values Heatmap")
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x="Brand", y="Price", data=df_train)
plt.title("Price Distribution by Brand")
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x="Material", y="Price", data=df_train)
plt.title("Price Distribution by Material")
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x="Size", y="Price", data=df_train)
plt.title("Price Distribution by Size")
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x="Color", y="Price", data=df_train)
plt.title("Price Distribution by Color")
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x="Style", y="Price", data=df_train)
plt.title("Price Distribution by Style")
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x="Laptop Compartment", y="Price", data=df_train)
plt.title("Price Distribution by Laptop Compartment")
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x="Waterproof", y="Price", data=df_train)
plt.title("Price Distribution by Waterproof")
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.scatterplot(x="Compartments", y="Price", data=df_train)
plt.title("Price Distribution by Compartments")
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.scatterplot(x="Weight Capacity (kg)", y="Price", data=df_train)
plt.title("Price Distribution by Weight Capacity (kg)")
plt.xticks(rotation=45)
plt.show()

## **Data Imputation**

**Brand, Material, Size, Laptop Compartment, Waterproof, Style, Color**
Due to the small catagorical types and little to interpret from the plots from the EDA the best approach for the missing values would be to use mode imputation

**Weight Capacity (kg)**
Here I will use the median value for the imputation since we are working with a continuous numerical variable.

In [ ]:
categorical_features = ["Brand", "Material", "Size", "Laptop Compartment", "Waterproof", "Style", "Color"]
numerical_features = ["Weight Capacity (kg)"]

for col in categorical_features:
    df_train[col].fillna(df_train[col].mode()[0], inplace=True)
    df_test[col].fillna(df_test[col].mode()[0], inplace=True)

for col in numerical_features:
    df_train[col].fillna(df_train[col].median(), inplace=True)
    df_test[col].fillna(df_test[col].median(), inplace=True)

## **Encoding Categorical Features**

In [ ]:
# One-Hot Encoding for categorical variables
one_hot_features = ["Brand", "Material", "Size", "Style", "Color"]
df_train = pd.get_dummies(df_train, columns=one_hot_features)
df_test = pd.get_dummies(df_test, columns=one_hot_features)


# Label Encoding for binary variables
binary_features = ["Laptop Compartment", "Waterproof"]
for col in binary_features:
    le = LabelEncoder()
    df_train[col] = le.fit_transform(df_train[col])
    
for col in binary_features:
    le = LabelEncoder()
    df_test[col] = le.fit_transform(df_test[col])

## **Baseline Model - Linear Regression**

In [ ]:
X = df_train.drop(columns=["Price", "id"])
y = df_train["Price"]


X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print("Baseline RMSE:", rmse)

## **LightGBM Model**

In [ ]:
X = df_train.drop(columns=["Price", "id"])
y = df_train["Price"]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

lgb_model = lgb.LGBMRegressor(
    n_estimators=1000, 
    learning_rate=0.05, 
    max_depth=10, 
    num_leaves=31
)
lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid)],
    eval_metric="rmse",
    callbacks=[lgb.early_stopping(100)]
)

y_pred = lgb_model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print("LightGBM RMSE:", rmse)


## **Hyperparameter Tuning**

In [ ]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 500, 2000),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.005, 0.1),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "num_leaves": trial.suggest_int("num_leaves", 20, 80),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 50),
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-3, 10.0),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-3, 10.0),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.5, 1.0),
        "subsample": trial.suggest_uniform("subsample", 0.5, 1.0),
    }
    

    model = lgb.LGBMRegressor(**params, random_state=42)
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric="rmse",
        callbacks=[lgb.early_stopping(10)],
    )
    
    y_pred = model.predict(X_valid)
    

    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
    
    return rmse

In [ ]:
early_stopping = MaxTrialsCallback(5)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)


print("Best Hyperparameters:", study.best_params)


In [ ]:
best_params = study.best_params

best_lgb_model = lgb.LGBMRegressor(**best_params, random_state=42)

best_lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid)],
    eval_metric="rmse",
    callbacks=[lgb.early_stopping(100)],
)

y_pred = best_lgb_model.predict(X_valid)

rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print("Optimized LightGBM RMSE:", rmse)


## **Submission File**

X_test = df_test.drop(columns=["id"])

y_test_pred = lr.predict(X_test)


# Create submission file
submission = pd.DataFrame({
    "id": df_test["id"],
    "Price": y_test_pred
})

# Step 4: Save to CSV (Ensure No Index)
submission.to_csv("submission.csv", index=False)

print("Submission file saved: submission.csv")

# # Rename Columns to Force LightGBM to Recognize Cleaned Names
X_train.columns = X_train.columns.str.replace(" ", "_").str.replace("(", "").str.replace(")", "")
X_valid.columns = X_valid.columns.str.replace(" ", "_").str.replace("(", "").str.replace(")", "")

# Confirm Feature Names
print("Feature Names in X_train:", X_train.columns.tolist())Ensure feature names are properly formatted in X_train, X_valid, and X_test
X_train.columns = X_train.columns.str.replace(" ", "_")
X_valid.columns = X_valid.columns.str.replace(" ", "_")
X_test.columns = X_test.columns.str.replace(" ", "_")

# Verify feature names
print("Feature Names in X_train:", X_train.columns.tolist())# Ensure feature names are properly formatted in X_train, X_valid, and X_test
X_train.columns = X_train.columns.str.replace(" ", "_")
X_valid.columns = X_valid.columns.str.replace(" ", "_")
X_test.columns = X_test.columns.str.replace(" ", "_")

# Verify feature names
print("Feature Names in X_train:", X_train.columns.tolist())X_test = df_test.drop(columns=["id"])
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)
y_test_pred = lgb_model.predict(X_test)

# Create Submission File
submission = pd.DataFrame({
    "id": df_test["id"],
    "Price": y_test_pred
})

submission.to_csv("submission.csv", index=False)
print("Submission file saved: submission.csv")
